# RULE BASED CLASSIFICATION

In [1]:
#Importing Libraries
import numpy as np
import pandas as pd
import seaborn as sns
df= pd.read_csv("/kaggle/input/persona/persona.csv")
df.head()

SyntaxError: unterminated string literal (detected at line 5) (1322254493.py, line 5)

## Recognizing the Dataset

In [3]:
df.isnull().sum()

,0
PRICE,0
SOURCE,0
SEX,0
COUNTRY,0
AGE,0


In [4]:
df.dtypes

,0
PRICE,int64
SOURCE,object
SEX,object
COUNTRY,object
AGE,int64


In [2]:
df["SOURCE"].value_counts()

,count
SOURCE,
android,2974
ios,2026


In [5]:
df["COUNTRY"].value_counts()

,count
COUNTRY,
usa,2065
bra,1496
deu,455
tur,451
fra,303
can,230


In [11]:
df.groupby("COUNTRY").agg({"PRICE":["sum", "mean"]})

PRICE           
           sum       mean
COUNTRY                  
bra      51354  34.327540
can       7730  33.608696
deu      15485  34.032967
fra      10177  33.587459
tur      15689  34.787140
usa      70225  34.007264

In [14]:
df.groupby(["COUNTRY", "SOURCE", "SEX", "AGE"])["PRICE"].mean()

COUNTRY  SOURCE   SEX     AGE
bra      android  female  15     38.714286
                          16     35.944444
                          17     35.666667
                          18     32.255814
                          19     35.206897
                                   ...    
usa      ios      male    42     30.250000
                          50     39.000000
                          53     34.000000
                          55     29.000000
                          59     46.500000
Name: PRICE, Length: 348, dtype: float64

## Aggregated Dataframe
We create a new dataframe that is regularized and aggregated.

In [15]:
agg_df= df.groupby(["COUNTRY", "SOURCE", "SEX", "AGE"])["PRICE"].mean().sort_values(ascending=False)
agg_df

COUNTRY  SOURCE   SEX     AGE
bra      android  male    46     59.0
usa      android  male    36     59.0
fra      android  female  24     59.0
usa      ios      male    32     54.0
deu      android  female  36     49.0
                                 ... 
usa      ios      female  38     19.0
                          30     19.0
can      android  female  27     19.0
fra      android  male    18     19.0
deu      android  male    26      9.0
Name: PRICE, Length: 348, dtype: float64

In [17]:
# We solve theindex problem.
agg_df= agg_df.reset_index()

In [18]:
# We create a new variable that separates consumers by age.
agg_df["AGE_CAT"]= pd.cut(agg_df["AGE"],[0, 18, 24, 30, 41, 70], labels=["0_18","19_23","24_30","31_40","41_70"])

In [19]:
# We create our variable that brings all the variables together. It resembles a decision tree.
# And we drop the initial variables.

agg_df["customers_level_based"]= (agg_df["COUNTRY"] + "_" + agg_df["SOURCE"] + "_" +
                                  agg_df["SEX"] + "_" +
                                  agg_df["AGE_CAT"].astype(str).str.replace("(","").str.replace("]","").str.replace(", ","_"))

agg_df.drop(["COUNTRY", "SOURCE", "SEX", "AGE_CAT", "AGE"], axis=1,inplace=True)

##New Dataframe

In [20]:
agg_df.groupby("customers_level_based")["PRICE"].mean()

,PRICE
customers_level_based,
bra_android_female_0_18,35.645303
bra_android_female_19_23,34.481117
bra_android_female_24_30,33.424603
bra_android_female_31_40,34.531899
bra_android_female_41_70,38.205128
...,...
usa_ios_male_0_18,33.983495
usa_ios_male_19_23,34.901872
usa_ios_male_24_30,34.838143


In [22]:
new_df = pd.DataFrame(data=agg_df, columns=["customers_level_based", "PRICE"])
new_df

,customers_level_based,PRICE
0,bra_android_male_41_70,59.0
1,usa_android_male_31_40,59.0
2,fra_android_female_19_23,59.0
3,usa_ios_male_31_40,54.0
4,deu_android_female_31_40,49.0
...,...,...
343,usa_ios_female_31_40,19.0
344,usa_ios_female_24_30,19.0
345,can_android_female_24_30,19.0
346,fra_android_male_0_18,19.0


In [24]:
new_df["Segment"]= pd.qcut(new_df["PRICE"], 4, labels=["D","C","B","A"])
new_df.groupby("Segment")["PRICE"].agg(["mean","max","sum"])

<ipython-input-24-e573c40f7e46>:2: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.

  new_df.groupby("Segment")["PRICE"].agg(["mean","max","sum"])


,mean,max,sum
Segment,,,
D,27.302596,31.105263,2375.325850
C,32.933339,34.000000,3128.667165
B,35.436170,37.000000,2870.329792
A,41.434736,59.000000,3521.952577


In [ ]:
## Predict

In [25]:
new_user = "tur_android_female_31_40"
new_user2 = "fra_ios_female_31_40"

In [31]:
new_df.loc[new_df["customers_level_based"] == new_user, "Segment"]

,Segment
18,A
35,A


In [32]:
new_df.loc[new_df["customers_level_based"] == new_user]["PRICE"].mean()

41.83333333333333


1. "tur_android_female_31_40" user is expected to be A segment.
2. "tur_android_female_31_40" user is expected to spend 41.833 dollars.

In [34]:
new_df.loc[new_df["customers_level_based"] == new_user2, "Segment" ]

,Segment
208,C
221,C


In [35]:
new_df.loc[new_df["customers_level_based"] == new_user2]["PRICE"].mean()

32.81818181818181


1. "fra_ios_female_31_40" user is expected to be A segment.
2. "fra_ios_female_31_40" user is expected to spend 32.818 dollars.